In [1]:
from pia_bench.bench import PiaBenchMark
from dotenv import load_dotenv
import os
load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")
model_name = "LLAVA"
model_name = "T2V_CLIP4Clip_KTT-UCFCrime_CW-pMSRVTT_20241120"
benchmark_path = "/mnt/nas_192tb/videos/huggingface_benchmarks_dataset/Leaderboard_bench/PIA"
cfg_target_path= "topk.json"

pia_benchmark = PiaBenchMark(benchmark_path ,model_name=model_name, cfg_target_path= cfg_target_path , token=access_token )
pia_benchmark.preprocess_structure()
pia_benchmark.preprocess_label_to_csv()  
print("Categories identified:", pia_benchmark.categories)

/home/piawsa6000/anaconda3/envs/bench/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Folder preprocessing completed.
All JSON files have already been processed to CSV. No further processing needed.
Categories identified: ['violence', 'fire', 'falldown']


In [ ]:
pia_benchmark.extract_visual_vector()

In [2]:
from pia_bench.event_alarm import EventDetector
detector = EventDetector(config_path=cfg_target_path, model_name=model_name , token=pia_benchmark.token)
detector.process_and_save_predictions(pia_benchmark.vector_video_path, 
                                      pia_benchmark.dataset_path, 
                                      pia_benchmark.alram_path)

/home/piawsa6000/work/seoik/Abnormal_situation_leader_board/DevMACS-AI-solution-devmacs/Package-Common-AI-pia_ai_package/packages/pia/ai/tasks/T2VRet/models/clip4clip/main.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for

In [2]:
from pia_bench.metric import MetricsEvaluator
metric = MetricsEvaluator(pred_dir=pia_benchmark.alram_path, 
                          label_dir=pia_benchmark.dataset_path, 
                          save_dir=pia_benchmark.metric_path)
metric.evaluate()


Category-wise Average Metrics:

fire:
falldown_f1: 0.000
falldown_accuracy: 0.004
falldown_precision: 0.000
falldown_recall: 0.000
falldown_specificity: 0.004
falldown_tp: 0.000
falldown_tn: 26.447
falldown_fp: 2551.763
falldown_fn: 0.000
violence_f1: 0.000
violence_accuracy: 0.732
violence_precision: 0.000
violence_recall: 0.000
violence_specificity: 0.732
violence_tp: 0.000
violence_tn: 1835.289
violence_fp: 742.921
violence_fn: 0.000
fire_f1: 0.811
fire_accuracy: 0.753
fire_precision: 0.753
fire_recall: 0.947
fire_specificity: 0.000
fire_tp: 954.816
fire_tn: 0.000
fire_fp: 1623.395
fire_fn: 0.000
category: fire

violence:
falldown_f1: 0.039
falldown_accuracy: 0.045
falldown_precision: 0.020
falldown_recall: 0.567
falldown_specificity: 0.025
falldown_tp: 178.033
falldown_tn: 153.467
falldown_fp: 6951.167
falldown_fn: 0.000
violence_f1: 0.233
violence_accuracy: 0.548
violence_precision: 0.247
violence_recall: 0.431
violence_specificity: 0.588
violence_tp: 621.633
violence_tn: 3654.63

{'micro_avg': {'tp': 66241,
  'tn': 334108,
  'fp': 1214564,
  'fn': 38366,
  'accuracy': 0.24215453048154606,
  'precision': 0.051718255315992674,
  'recall': 0.633236781477339,
  'f1': 0.09562642737322904},
 'falldown': {'tp': 11309,
  'tn': 6667,
  'fp': 533117,
  'fn': 0,
  'accuracy': np.float64(0.0326188138844079),
  'precision': np.float64(0.020772336368946376),
  'recall': np.float64(1.0),
  'specificity': np.float64(0.012351236791012701),
  'f1': np.float64(0.04069925414091249),
  'balanced_accuracy': np.float64(0.5061756183955064),
  'g_mean': np.float64(0.11113611830099475),
  'mcc': np.float64(0.01601761671396284),
  'npv': np.float64(1.0),
  'far': np.float64(0.9876487632089873)},
 'violence': {'tp': 18649,
  'tn': 327396,
  'fp': 166682,
  'fn': 38366,
  'accuracy': np.float64(0.6279248693051808),
  'precision': np.float64(0.10062536758556313),
  'recall': np.float64(0.32708936244847847),
  'specificity': np.float64(0.662640311853594),
  'f1': np.float64(0.153903922490983